In [1]:
#!/usr/bin/python
from __future__ import absolute_import

import numpy as np
from tensorflow import keras
import pickle
import argparse
import copy
import random

from shapley.apps import Label, Poisoning
from shapley.loader import FashionMnist, MNIST
from shapley.measures import KNN_Shapley, KNN_LOO, G_Shapley, LOO, TMC_Shapley
from shapley.utils.plotter import LabelPlotter, PoisoningPlotter

%matplotlib inline

/home/zidaneandmessi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zidaneandmessi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zidaneandmessi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zidaneandmessi/.local/lib/python3.6/site-packages/tensorflow/

# Noisy Label Detection

##### data loading

In [ ]:
num = 1000
#loader = MNIST(num_train=num, one_hot=False, shuffle=True, by_label=True)
loader = FashionMnist(num_train=num)
X_data, y_data, X_test_data, y_test_data = loader.prepare_data()

###### measure

In [ ]:
measure_KNN = KNN_Shapley(K=5)
measure_KNNLOO = KNN_LOO(K=5)
# measure = G_Shapley()
# measure = TMC_Shapley()
# measure = LOO()

##### application

In [ ]:
app = Label(X_data, y_data, X_test_data, y_test_data, model_family='NN')

##### run

In [ ]:
res_knn = app.run(measure_KNN)
res_knnloo = app.run(measure_KNNLOO)

##### visualize

In [ ]:
LabelPlotter(app, ('KNN-Shapley',res_knn), ('KNN-LOO',res_knnloo)).plot()

# Pattern-based Watermark Removal

##### data loading

In [2]:
num = 1000
loader = FashionMnist(num_train=num)
X_data, y_data, X_test_data, y_test_data = loader.prepare_data()

###### measure

In [3]:
measure_KNN = KNN_Shapley(K=5)
measure_KNNLOO = KNN_LOO(K=5)

##### application

In [4]:
app = Poisoning(X_data, y_data, X_test_data, y_test_data, model_family='NN')

##### run

In [ ]:
res_knn = app.run(measure_KNN)
res_knnloo = app.run(measure_KNNLOO)

done!
result shown below:
[ 9.57917193e-04  9.32139859e-04  1.06305893e-03  1.76369357e-04
  8.89899280e-04 -7.04438610e-04  7.05498645e-04  1.49505371e-04
  8.55515620e-04  1.45748081e-03  1.16173499e-03  6.99311700e-04
  6.79187235e-04  6.38301575e-04  1.21188007e-03  1.08445871e-03
  1.05476239e-03 -4.16956496e-04  1.13673073e-03  8.44427549e-04
  9.46335966e-04 -1.59260120e-03  1.08537192e-03  1.28756606e-03
  7.12005658e-04 -6.85083598e-03  2.02937971e-03  9.80251426e-04
  1.13986219e-03  1.15305328e-03  9.81813475e-04  1.02911247e-03
  7.78581030e-04 -1.36602499e-04  8.27937129e-04  8.99714780e-04
  2.21653193e-04  1.80692215e-03  3.82705059e-04  2.62721989e-04
  6.56044568e-04 -3.00717288e-03  1.19363620e-03  7.39993739e-04
 -3.00036536e-04  9.72498947e-04  1.03915565e-03  1.88682334e-03
  1.01621750e-03  4.61719143e-04  5.18072098e-04  7.08169333e-04
  8.24795753e-04  1.31385443e-03  8.71812581e-04  1.22596713e-03
  1.59904850e-03  2.31598413e-03  8.83994858e-04  1.07610560e-03

##### visualize

In [ ]:
PoisoningPlotter(app, ('KNN-Shapley',res_knn), ('KNN-LOO',res_knnloo)).plot()

# Instance-based Watermark Removal

##### data loading

In [ ]:
# load MNIST

###### measure

In [ ]:
measure = KNN_Shapley(K=5)

##### application

In [ ]:
app = # instance-based watermark removal

##### run

In [ ]:
# run

##### visualize

In [ ]:
# TO BE COMPLETED

# Data Summarization

##### data loading

In [ ]:
# load UCI Adult dataset

###### measure

In [ ]:
measure = KNN_Shapley(K=5)

##### application

In [ ]:
app = # data summarization

##### run

In [ ]:
# run

##### visualize

In [ ]:
# TO BE COMPLETED

# Data Acquisition

##### data loading

In [ ]:
# load MNIST

###### measure

In [ ]:
measure = KNN_Shapley(K=5)

##### application

In [ ]:
app = # data acquisition

##### run

In [ ]:
# run

##### visualize

In [ ]:
# TO BE COMPLETED

# Domain Adapatation

##### data loading

In [ ]:
# load two datasets

###### measure

In [ ]:
measure = KNN_Shapley(K=5)

##### application

In [ ]:
app = # domain adaptation

##### run

In [ ]:
# run

##### visualize

In [ ]:
# TO BE COMPLETED